In [1]:
import ujson
import csv
import pandas as pd
import numpy as np

from Scripts import TOOI_organisaties

organisaties = TOOI_organisaties(orgs=['provincies'])

In [2]:
from collections import Counter

In [3]:
with open(f"SFTP_beschikkingen_wsb.json", "r") as file:
    beschikkingen = ujson.load(file)

In [4]:
# Tel alle keren dat een bepaald GEOattribuut voorkomt zodat we ook hiermee kunnen analyseren

alle_beschikkingen = {}
counter = 0
for k,v in beschikkingen.items():
    for c, i in enumerate(v):
        doc = i['infobox']
        if not isinstance(beschikkingen[k][c]['infobox']['foi_files'], int):
            beschikkingen[k][c]['infobox']['foi_files'] = len(doc['foi_files']['foi_documenten'])
        if 'foi_overheidOP_geo' in doc:
            if doc['foi_overheidOP_geo'] != []:
                count_dict = {}
                for i in doc['foi_overheidOP_geo']:
                    for KEY,VALUE in i.items():
                        if KEY not in count_dict.keys():
                            count_dict[KEY] = 1
                        else:
                            count_dict[KEY]+=1
    
                for KEY2,VALUE2 in count_dict.items():
                    beschikkingen[k][c]['infobox'][KEY2] = VALUE2      
                        
            beschikkingen[k][c]['infobox'].pop('foi_overheidOP_geo')

        alle_beschikkingen[counter] = beschikkingen[k][c]['infobox']
        counter+=1

In [5]:
df = pd.DataFrame(alle_beschikkingen).T

In [6]:
display(df.head())

,dc_title,dc_date_year,foi_publishedDate,dc_source,tooiwl_topic,dc_publisher_name,foi_startDate,foi_endDate,tooiwl_rubriek,dc_type,...,OVERHEIDop.huisletter,kml_LineString,OVERHEIDop.geometrieLabel,kml_Polygon,OVERHEIDop.waterschapsnaam,OVERHEIDop.huisnummerToevoeging,OVERHEIDop.effectgebied,Test_Terinlagelegging,OVERHEIDop.kadastraleSectie,OVERHEIDop.perceelnummer
0,Aanvraag watervergunning vernieuwen remmingswe...,2013,05-03-2013,https://zoek.officielebekendmakingen.nl/wsb-20...,Natuur en milieu | Water,Waterschap Brabantse Delta,2013-02-28,2013-03-28,Beschikkingen | aanvraag,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Verleende vergunning 13UT002188 ingevolge de K...,2013,2013-03-11,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-02-26,2013-04-09,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Verleende vergunning 13UT006559 ingevolge de K...,2013,2013-06-13,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-05-21,2013-07-02,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Verleende vergunning 13UT006525 ingevolge de K...,2013,2013-06-13,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-05-21,2013-07-02,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Verleende vergunning 13UT006589 ingevolge de K...,2013,13-06-2013,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-05-22,2013-07-03,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Een overzicht van alle kolommen die we hebben
print(df.columns)
print('')
print(Counter(df['dc_date_year']))

Index(['dc_title', 'dc_date_year', 'foi_publishedDate', 'dc_source',
       'tooiwl_topic', 'dc_publisher_name', 'foi_startDate', 'foi_endDate',
       'tooiwl_rubriek', 'dc_type', 'dc_publisher', 'dc_description',
       'foi_files', 'foi_betreftRegeling', 'tooiwl_rubriek_code',
       'OVERHEIDop.gebiedsmarkering', 'kml_Point', 'kml_coordinates',
       'OVERHEIDop.ligtInGemeente', 'OVERHEIDop.huisnummer',
       'OVERHEIDop.postcode', 'OVERHEIDop.straatnaam', 'OVERHEIDop.woonplaats',
       'OVERHEIDop.huisletter', 'kml_LineString', 'OVERHEIDop.geometrieLabel',
       'kml_Polygon', 'OVERHEIDop.waterschapsnaam',
       'OVERHEIDop.huisnummerToevoeging', 'OVERHEIDop.effectgebied',
       'Test_Terinlagelegging', 'OVERHEIDop.kadastraleSectie',
       'OVERHEIDop.perceelnummer'],
      dtype='object')

Counter({'2021': 15483, '2020': 14323, '2022': 14172, '2019': 13225, '2023': 12421, '2018': 12339, '2017': 11351, '2016': 8870, '2015': 8224, '2014': 7677, '2013': 3653})


In [9]:
grouped_data = df.groupby(['dc_date_year', 'dc_publisher_name'])['dc_title'].count().reset_index()

pivot_table = pd.pivot_table(grouped_data, values='dc_title', index='dc_date_year', columns='dc_publisher_name', fill_value=0).astype(int).sort_index(ascending=False).T
pivot_table['Totaal'] = pivot_table.sum(axis=1)
pivot_table.loc['Aantal per Jaar'] = pivot_table.sum(axis=0)

display(pivot_table)

dc_date_year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,Totaal
dc_publisher_name,,,,,,,,,,,,
Hoogheemraadschap De Stichtse Rijnlanden,1247,1424,1244,693,798,1163,993,821,853,800,80,10116
Hoogheemraadschap Hollands Noorderkwartier,1158,1524,1957,2031,2092,1641,1707,1364,1493,1324,7,16298
Hoogheemraadschap van Delfland,1337,1027,1844,1268,1048,943,784,889,671,66,0,9877
Hoogheemraadschap van Rijnland,691,883,861,907,742,814,913,761,901,1136,837,9446
Hoogheemraadschap van Schieland en de Krimpenerwaard,829,1055,1029,1045,795,747,804,805,747,672,247,8775
Waterschap Aa en Maas,371,368,478,394,332,225,228,18,0,0,0,2414
"Waterschap Amstel, Gooi en Vecht",565,638,743,726,928,768,620,0,0,0,0,4988
Waterschap Brabantse Delta,787,1027,1549,2411,1883,1765,964,677,580,939,892,13474
Waterschap De Dommel,474,328,345,382,327,281,267,8,0,0,0,2412


Als eerste tabel volgen de tellingen van het aantal publicaties per waterschap per jaar. Zoals meestal het geval is zijn bepaalde organisaties eerder begonnen met het publiceren. Dit kan hier door twee redenen komen: 1. Het waterschap is pas in de recentste jaren opgericht (bijvoorbeeld Waterschap Limburg wat op 1 januari 2017 is ontstaan) en daarna is altijd gepubliceerd. 2. Het waterschap is pas later begonnen met publiceren. Zoals bijvoorbeeld Hunze en Aa's en Amstel, Gooi en Vecht. 

Zoals vverwacht neemt het aantal publicaties met de jaren toe. Tot dat het rond 2021 stabiliseerd. Waarschijnlijk worden vanaf dit moment alle mogelijke publicaties ook echt openbaar gemaakt en is het getal hierna vooral afhankelijk van het aantal daadwerkelijk bestaande documenten. 

**Opvallende zaken individuele waterschappen**
- Scheldestromen: Hier lijkt het aantal publicaties pas halverwege 2021 ineens erg toegenomen. Wellicht werden hiervoor de beschikkingen nog niet gepubliceerd maar wel een aantal van de andere rubrieksoorten.

In [10]:
grouped_data = pd.DataFrame(df.groupby(['dc_publisher_name'])['foi_files'].sum())
grouped_data['totaal'] = pivot_table['Totaal']
grouped_data.loc['Totaal alle organisaties'] = grouped_data.sum(axis=0)
grouped_data['files/org'] = grouped_data['foi_files']/grouped_data['totaal']

display(grouped_data)

,foi_files,totaal,files/org
dc_publisher_name,,,
Hoogheemraadschap De Stichtse Rijnlanden,19044,10116,1.882562
Hoogheemraadschap Hollands Noorderkwartier,17906,16298,1.098662
Hoogheemraadschap van Delfland,21698,9877,2.196821
Hoogheemraadschap van Rijnland,17563,9446,1.859306
Hoogheemraadschap van Schieland en de Krimpenerwaard,9733,8775,1.109174
Waterschap Aa en Maas,4333,2414,1.794946
"Waterschap Amstel, Gooi en Vecht",17451,4988,3.498597
Waterschap Brabantse Delta,31033,13474,2.303176
Waterschap De Dommel,4489,2412,1.861111


De de volgende tabel staan het aantal bijlagen per organisatie en daarmee het gemiddeld aantal bijlagen per publicatie weergegeven. Hierin valt op dat er grote verschillen zijn in de mate van het publiceren van bijlagen. Een aaantal organisaties lijken hiermee heel consequent te zijn zoals Waterschap Limburg en Waterschap AGV die gemiddeld meerdere bijlagen per publicatie hebben. Terwijl er ook organisatie zijn waarbij een bijlage vrij zelfdzaam is zoals bijvoorbeeld Scheldestromen en Rivierenland. Het grootste deel van de waterschappen zit er een beetje tussen in wat ook wel verklaard dat er gemiddeld 0.8 bijlagen zijn per publicatie. 

Het is echter niet waarschijnlijk dat dit betekent dat bijna 80% van de publicaties bijlagen heeft. Waarschijnlijk is dit een veel lager percentage. 

In [11]:
foi_files_counter = Counter(df['foi_files'])
print(foi_files_counter)

print(foi_files_counter[0] / sum(foi_files_counter.values()))

Counter({1: 72454, 2: 33238, 3: 7600, 4: 3515, 5: 1968, 6: 895, 7: 506, 8: 301, 9: 242, 10: 200, 11: 130, 12: 105, 13: 86, 14: 83, 15: 51, 16: 48, 17: 32, 19: 30, 23: 25, 20: 22, 22: 20, 21: 19, 18: 18, 24: 15, 27: 14, 28: 10, 26: 10, 37: 9, 32: 9, 25: 8, 38: 7, 35: 6, 39: 6, 29: 5, 31: 5, 34: 4, 33: 3, 101: 3, 40: 3, 66: 2, 51: 2, 63: 2, 30: 2, 48: 1, 68: 1, 57: 1, 261: 1, 121: 1, 102: 1, 91: 1, 93: 1, 140: 1, 257: 1, 313: 1, 77: 1, 97: 1, 96: 1, 76: 1, 52: 1, 174: 1, 41: 1, 69: 1, 54: 1, 70: 1, 84: 1, 75: 1, 49: 1, 482: 1})
0.0


Door het uitvoeren van de Counter() python functie kan worden bepaald welk percentage van de publicaties helemaal geen bijlagen bevat. Het blijkt dus dat dit inderdaad een stuk meer is dan het totaal zou laten zien. Het blijkt dat slechts 40.5 procent van de publicaties maar minimaal 1 bijlage heeft. 

Wat ik zelf ook nog interessant vind om naar te kijken zijn extreme waarden. Zijn er bijvoorbeeld publicaties met een extreem hoog aantal bijlagen? Natuurlijk kan dit ook al blijken uit de counter.

In [12]:
print(f"{len(df[df['foi_files'] > 5])} publicaties met meer dan 5 bijlagen.")
print(f"{len(df[df['foi_files'] > 10])} publicaties met meer dan 10 bijlagen.")
print(f"{len(df[df['foi_files'] > 25])} publicaties met meer dan 25 bijlagen.")
print(f"{len(df[df['foi_files'] > 50])} publicaties met meer dan 50 bijlagen.")
print(f"{len(df[df['foi_files'] > 100])} publicaties met meer dan 100 bijlagen.")

result = df[df['foi_files'] > 480]
for i in result.dc_source:
    print(i)

2963 publicaties met meer dan 5 bijlagen.
819 publicaties met meer dan 10 bijlagen.
127 publicaties met meer dan 25 bijlagen.
31 publicaties met meer dan 50 bijlagen.
11 publicaties met meer dan 100 bijlagen.
https://zoek.officielebekendmakingen.nl/wsb-2023-3911


Het blijkt dus dat het vrij zeldzaam is voor een publicatie om meer dan 5 bijlagen te hebben. Dit is in flink minder dan 2 procent van de gevallen zo. En het worden er seeds minder naarmate het aantal bijlagen toeneemt. Toch zijn er nog 8 publicaties met meer dan 100 bijlagen. En is er zelfs een extreme outlier met 481 bijlagen. De meeste van deze outliers komen van het waterschap brabantse delta. Al is dit wel één van de waterschappen met de meeste publicaties. 

In [13]:
grouped_data = pd.DataFrame(df.groupby(['dc_date_year'])['foi_files'].sum())
grouped_data['totaal'] = df.groupby(['dc_date_year'])['foi_files'].count()

grouped_data.loc['Totaal alle jaren'] = grouped_data.sum(axis=0)
grouped_data['files/org'] = grouped_data['foi_files']/grouped_data['totaal']

display(grouped_data)

,foi_files,totaal,files/org
dc_date_year,,,
2013,6040,3653,1.653436
2014,11646,7677,1.516999
2015,13367,8224,1.625365
2016,14530,8870,1.638106
2017,21889,11351,1.928376
2018,25652,12339,2.078937
2019,25876,13225,1.956597
2020,29587,14323,2.065699
2021,29499,15483,1.905251


Hier is het aantal bijlagen per publicatie gegroepeerd per jaar. Het valt hier op dat in de eerste vier jaren de waarde gemiddeld rond de 0.6 bijlagen per publicatie lag, om vervolgens vanaf 2017 te stijgen naar ongeveer 1 bijlage per publicatie en dit werd volgehouden tot en met 2021. Waarna Het aantal bijlagen aanzienlijk is gedaal naar tot onder het niveau van 2013. Dit lijkt dus geen goede ontwikkeling te zijn. Wellicht dat de terinzagelegginen hiervan de oorzaak zijn en dat de organisaties denken dat de bijlagen niet meer gepubliceerd moeten worden als ze al in de terinzagelegging komen. Met als nadeel natuurlijk dat de documenten maar voor een paar maanden terug te vinden zijn.

Edit: Bij de waterschappen zijn bijna geen terinzageleggingen. Dus ik noem het nalatigheid! (het zijn er wellicht 20 in totaal)


In [14]:
GEO_groupby = df.groupby(['dc_publisher_name'])[['kml_Point', 'kml_Polygon', 'kml_LineString']].sum()
GEO_groupby.loc['Totaal aantal per type'] = GEO_groupby.sum(axis=0)
display(GEO_groupby)

,kml_Point,kml_Polygon,kml_LineString
dc_publisher_name,,,
Hoogheemraadschap De Stichtse Rijnlanden,3033,376,610
Hoogheemraadschap Hollands Noorderkwartier,4922,50,10
Hoogheemraadschap van Delfland,3582,8,267
Hoogheemraadschap van Rijnland,1470,151,488
Hoogheemraadschap van Schieland en de Krimpenerwaard,1299,799,1820
Waterschap Aa en Maas,818,56,50
"Waterschap Amstel, Gooi en Vecht",1931,151,41
Waterschap Brabantse Delta,2068,80,704
Waterschap De Dommel,843,57,14


Dit is het aantal keer dat een bepaald soort geolocatie voorkomt per organisatie. Het meest voorkomende soort geolocatie is een point, daarna een linestring en daarna een polygon. Eigenlijk vrij logisch dus. Het valt wel op dat een aantal organisaties wel een iets afwijkende verdeling hebben. Bijvoorbeeld wetterskip fryslan heeft verwassend veel polygons en linestrings vergeleken met het gemiddelde. Hetzelfde geld ook wel voor Rijn en IJssel en Schieland en de Krimpenewaard.

Deze getallen vallen natuurlijk veel lager uit dan het totaal aantal publicaties. Dit komt omdat alleen het nieuwe geo-systeem is meegenomen. Vandaar dat het geen enkel nut heeft om te groeperen voor jaren voor 2021. 

In [15]:
GEO_groupby = df.groupby(['dc_date_year'])[['OVERHEIDop.gebiedsmarkering', 'kml_Point',
       'kml_coordinates', 'OVERHEIDop.ligtInGemeente', 'OVERHEIDop.huisnummer',
       'OVERHEIDop.postcode', 'OVERHEIDop.straatnaam', 'OVERHEIDop.woonplaats',
       'OVERHEIDop.huisletter', 'kml_LineString', 'OVERHEIDop.geometrieLabel',
       'kml_Polygon', 'OVERHEIDop.waterschapsnaam',
       'OVERHEIDop.huisnummerToevoeging', 'OVERHEIDop.effectgebied',
       'OVERHEIDop.kadastraleSectie', 'OVERHEIDop.perceelnummer']].sum()

GEO_groupby.index = pd.to_numeric(GEO_groupby.index)
GEO_groupby = GEO_groupby[GEO_groupby.index >= 2021]

display(GEO_groupby.T)

dc_date_year,2021,2022,2023
OVERHEIDop.gebiedsmarkering,9105,17388,19166
kml_Point,6894,12321,14016
kml_coordinates,6894,12321,14016
OVERHEIDop.ligtInGemeente,4470,8137,7445
OVERHEIDop.huisnummer,3053,5490,5513
OVERHEIDop.postcode,3016,5425,5456
OVERHEIDop.straatnaam,3053,5490,5513
OVERHEIDop.woonplaats,3053,5490,5513
OVERHEIDop.huisletter,388,731,858
kml_LineString,1231,2701,2892


Hier een overzicht van alle Geo attributen die verzameld kunnen worden. Het grootste deel hiervan zijn dingen die we mogen verwachten zoals adressgegevens. Maar er zijn ook een aantal obscure attributen: Dit zijn effectgebied, kadastraleSectie en perceelnummer (waarbij de laatste twee dan weer bij elkaar horen). Hetzelfde geld eigenlijk ook voor huisnummer, straatnaam en woonplaats. Deze horen ook altijd bij elkaar. 

Ik heb alleen geen idee wat het verschil tussen huisletter en huisnummerToevoeging is dus dit zal ik nog onderzoeken! **Op basis van wat ik weet is het niet hetzelfde, maar ik heb ook niet echt kunnen ontdekken wat de toevoeging precies betekent!**

In [16]:
result = df[df['OVERHEIDop.gebiedsmarkering'] > 0]
result1 = result.groupby('dc_publisher_name')['OVERHEIDop.gebiedsmarkering'].sum()
result2 = result.groupby('dc_publisher_name')['OVERHEIDop.gebiedsmarkering'].count()
res_df = pd.DataFrame({'Aantal Gebiedsmarkeringen': result1, 'Aantal Publicaties': result2})
res_df['Geo per Publicatie'] = res_df['Aantal Gebiedsmarkeringen'] / res_df['Aantal Publicaties']
display(res_df)

,Aantal Gebiedsmarkeringen,Aantal Publicaties,Geo per Publicatie
dc_publisher_name,,,
Hoogheemraadschap De Stichtse Rijnlanden,4100,3394,1.208014
Hoogheemraadschap Hollands Noorderkwartier,5074,3643,1.392808
Hoogheemraadschap van Delfland,3935,2857,1.377319
Hoogheemraadschap van Rijnland,2176,1974,1.10233
Hoogheemraadschap van Schieland en de Krimpenerwaard,4066,2360,1.722881
Waterschap Aa en Maas,1104,964,1.145228
"Waterschap Amstel, Gooi en Vecht",2228,1565,1.423642
Waterschap Brabantse Delta,3057,2489,1.228204
Waterschap De Dommel,1160,977,1.187308


Het valt dus op dat het aantal geolocaties per publicatie (voor de publicaties met minimaal 1 geolocatie) altijd lager ligt dan 2. Al zijn er ook organisaties die bijna nooit meer dan 1 geolocaties hebben zoals Vechtstromen en Noorderzijlvest. Terwijl Zuiderzeeland en Wetterskip Fryslan bijna twee geolocaties hebben. Dit is eigenlijk best vreemd want je zou dan verwachten dat de waterschappen met weinig geolocaties dan vaker voor een linestring of polygon zouden kiezen, maar dit blijkt niet het geval te zijn want dit komt juist veel vaker voor bij wetterskip fryslan. 

Omdat geolocaties een oneindig lange lijst kunnen worden ben ik ook nog geinteresseerd in de extreme waardes hier. Zijn er publicaties met een heel hoog aantal geolocaties?

In [17]:
foi_files_counter = Counter(df['OVERHEIDop.gebiedsmarkering'])
print(foi_files_counter)

print(foi_files_counter[0] / sum(foi_files_counter.values()))

Counter({nan: 87688, 1: 28878, 2: 3085, 3: 979, 4: 519, 5: 138, 6: 85, 10: 64, 7: 61, 8: 45, 9: 40, 13: 26, 12: 25, 11: 24, 17: 16, 14: 12, 15: 11, 18: 8, 25: 6, 16: 5, 22: 5, 20: 4, 21: 2, 38: 2, 36: 2, 56: 1, 54: 1, 19: 1, 109: 1, 37: 1, 23: 1, 28: 1, 24: 1})
0.0


In [18]:
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 5])} publicaties met meer dan 5 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 10])} publicaties met meer dan 10 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 25])} publicaties met meer dan 25 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 50])} publicaties met meer dan 50 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 100])} publicaties met meer dan 100 locaties.")

result = df[df['OVERHEIDop.gebiedsmarkering'] > 108]
for i in result.dc_source:
    print(i)

451 publicaties met meer dan 5 locaties.
156 publicaties met meer dan 10 locaties.
9 publicaties met meer dan 25 locaties.
3 publicaties met meer dan 50 locaties.
1 publicaties met meer dan 100 locaties.
https://zoek.officielebekendmakingen.nl/wsb-2022-8219


Het is hier toch wel vrij zeldzaam dat een publicatie meer dan 5 geolocaties heeft. Dit gebeurt eigenlijk maar in minder dan een fractie van de gevallen. En meer dan 25 locaties is al helemaal zeldzaam. Er is hier 1 uitschieter met 109 locaties. Dit zijn eigenlijk een groot aantal plaatsen in de achterhoek. 

In [19]:
print(Counter(df['tooiwl_rubriek_code'])[np.nan] / sum(Counter(df['tooiwl_rubriek_code']).values()))

0.7203009742233321


Helaas is de tooi_rubriek code niet echt rendabel om te gebruiken wegens het grote aantal rubrieken zonder tooi code. Hoewel ik dus vermoedde dat dit wel zou kunnen voor meer recentere gevallen. Dat ga ik nu onderzoeken!

In [20]:
result_rubriek = df.groupby('dc_date_year')['tooiwl_rubriek_code'].count()

result_gebied = df[df['OVERHEIDop.gebiedsmarkering'] != np.nan]
result_gebied = result_gebied.groupby('dc_date_year')['OVERHEIDop.gebiedsmarkering'].count()

result_df = pd.DataFrame({'TOOI_rubriek_code': result_rubriek, 'GeoLocaties': result_gebied})
display(result_df)

,TOOI_rubriek_code,GeoLocaties
dc_date_year,,
2013,0,0
2014,0,0
2015,0,0
2016,0,0
2017,0,0
2018,0,0
2019,0,0
2020,0,0
2021,7457,7457


Tijdens dit onderzoekje bleek ten eerste dat dit ook echt klopt dat dus in de laatste paar jaar dit wel consitent werkt met deze TOOI. Maar verder viel mij nog wat op. Het aantal documenten met de nieuwe TOOI bleek nogal overeen te komen met het patroon wat ik had gevonden voor de geolocaties. En toen ik deze erbij haalde bleek dit zelfs een honder procent match te zijn. Dit kan geen toeval zijn en het lijkt dus dat deze veranderingen tegelijkertijd zijn ingevoerd!

In [21]:
rubriek_counter = Counter(df['tooiwl_rubriek'])
display(rubriek_counter)

Counter({'Beschikkingen | afhandeling': 62188,
         'Beschikkingen | aanvraag': 13352,
         'vergunning voor activiteiten op of in oppervlaktewater': 11103,
         'andere vergunning': 10330,
         'omgevingsvergunning': 4962,
         'Overige overheidsinformatie': 4803,
         'Overige besluiten van algemene strekking': 4459,
         'andere beschikking': 2771,
         'Plannen | overig': 1394,
         'Verordeningen': 1167,
         'andere melding': 1105,
         'ander besluit van algemene strekking': 1076,
         'overige overheidsinformatie': 1073,
         'algemeen verbindend voorschrift (verordening)': 383,
         'melding van activiteiten op of in oppervlaktewater': 318,
         'Beleidsregels': 302,
         'omgevingsmelding': 291,
         'beleidsregel': 122,
         'vergadering of stuk van het bestuur': 107,
         'andere voorlichtingsinformatie': 94,
         'verkeersbesluit of -mededeling': 73,
         'participatie': 72,
         'inspr

Omdat we de publicaties graag willen classificeren in de verschillende informatiesoorten is het niet onbelangrijk om ook de rubrieknamen onder de loep te nemen. Hier valt al gelijk op dat de beschikkingen onder het oude systeem veruit het meeste voorkomen. Ik zal hieronder proberen om alle rubrieknamen te classificeren onder bepaalde informatietypen.

In [22]:
beschikkingen = []
besluiten_van_alg = []
meldingen = []
overigen = []
voorlichtingen = []

for k in rubriek_counter:
    if 'beschikking' in k.lower() or 'vergunning' in k.lower():
        beschikkingen.append(k)
    elif k in ['algemeen verbindend voorschrift (verordening)', 'Overige besluiten van algemene strekking', 'beleidsregel','Beleidsregel', 'Beleidsregels', 'delegatie- of mandaatbesluit','bekendmaking aan de scheepvaart', 'algemeenverbindendverklaring van cao-bepalingen', 'gemeenschappelijke regeling', 'ruimtelijk plan of omgevingsdocument','verkeersbesluit of -mededeling', 'ander besluit van algemene strekking']:
        besluiten_van_alg.append(k)
    elif 'plan' in k.lower() or 'verordening' in k.lower():
        besluiten_van_alg.append(k)
    elif 'melding' in k.lower():
        meldingen.append(k)
    elif k in ['vergadering of stuk van het bestuur', 'verkiezingen', 'andere voorlichtingsinformatie', 'inspraak', 'openingstijden', 'afvalinzameling']:
        voorlichtingen.append(k)
    else:
        overigen.append(k)


display(beschikkingen)
print('')
display(besluiten_van_alg)
print('')
display(meldingen)
print('')
display(voorlichtingen)
print('')
display(overigen)

['Beschikkingen | aanvraag',
 'Beschikkingen | afhandeling',
 'andere vergunning',
 'vergunning voor activiteiten op of in oppervlaktewater',
 'andere beschikking',
 'exploitatievergunning',
 'omgevingsvergunning',
 'evenementenvergunning']

['Beleidsregels',
 'Plannen | ruimtelijk',
 'Plannen | overig',
 'Overige besluiten van algemene strekking',
 'Verordeningen',
 'ander besluit van algemene strekking',
 'algemeen verbindend voorschrift (verordening)',
 'verkeersbesluit of -mededeling',
 'beleidsregel',
 'delegatie- of mandaatbesluit',
 'gemeenschappelijke regeling',
 'ruimtelijk plan of omgevingsdocument']

['melding van activiteiten op of in oppervlaktewater',
 'andere melding',
 'omgevingsmelding']

['inspraak',
 'vergadering of stuk van het bestuur',
 'andere voorlichtingsinformatie',
 'verkiezingen']

['Overige overheidsinformatie', 'overige overheidsinformatie', 'participatie']

Dit is een beetje hoe ik dit in zou delen. Waarvan de eerste twee dus zeker een infomatiecategorie zijn die onder de woo valt. Die moeten dus zeker verzameld worden, wat er met de andere drie moet gebeuren laat ik in het midden!

In [23]:
rubriek_counter = Counter(df['tooiwl_topic'])
display(rubriek_counter)

Counter({'Ruimte en infrastructuur | Organisatie en beleid': 57633,
         'Natuur en milieu | Organisatie en beleid': 22809,
         'Bestuur | Organisatie en beleid': 18218,
         'Ruimte en infrastructuur | Waterkeringen en waterbeheer': 8642,
         'Bestuur | Waterschappen': 6728,
         'Natuur en milieu | Water': 2383,
         'Recht | Organisatie en beleid': 1334,
         'Verkeer | Organisatie en beleid': 1111,
         'Financiën | Organisatie en beleid': 1060,
         'Cultuur en recreatie | Organisatie en beleid': 613,
         'Verkeer | Weg': 226,
         'Recht | Bezwaar en klachten': 148,
         'Landbouw | Organisatie en beleid': 122,
         'Openbare orde en veiligheid | Organisatie en beleid': 98,
         'Recht | Burgerlijk recht': 85,
         'Ruimte en infrastructuur | Ruimtelijke ordening': 83,
         'Financiën | Belasting': 74,
         'Natuur en milieu | Bodem': 60,
         'Natuur en milieu | Stoffen': 50,
         'Economie | Organisa

Dit is een counter van de tooiwl_topics. Je ziet dus dat er een aantal heel vaak voorkomen, maar dat een vrij groot deel extreem zeldzaam is. Hoewel ik hierdoor ook een fout heb ontdekt in mijn json. Want volgens mij kunnen publicaties meerdere van deze topics hebben, maar heb ik er maximaal 1. Dus dit zal ik nog aanpassen in de toekomst! Hoewel ik nog niet precies weet hoe dit geimplementeerd moet worden.

Omdat ik de startdate en enddate nogal vaag vind ga ik deze ook nog even kort bekijken. Met de `foi_betreftRegeling` want daar horen ze volgens mij bij!

In [24]:
res_dates = df.groupby('dc_date_year')[['foi_startDate', 'foi_endDate', 'foi_betreftRegeling']].count()
display(res_dates)

,foi_startDate,foi_endDate,foi_betreftRegeling
dc_date_year,,,
2013,1782,1641,0
2014,3199,3017,0
2015,3491,3311,0
2016,29,0,29
2017,115,6,115
2018,219,9,219
2019,300,11,309
2020,224,14,233
2021,258,14,266


De hoop dat het hierdoor minder vaag zou worden is helaas geen werkelijkheid geworden. Het lijkt erop dat de startDate en endDate iets anders betekende in de eerste drie jaar dan in de overige jaren. Ik wil best geloven dat het vanaf 2016 een begin en eindatum zijn van een regeling. (Dat lijkt wel zeker voor 2016, 2017 en 2018 waar de cijfers perfect overeenkomen). En ook wel aannemelijk voor de jaren hiernaa waar soms wel een startdatum mist, maar de cijfers alsnog best goed overeenkomen. En dan de einddatum als een regeling ook echt tot een einde is gekomen. (Dit zal vast niet altijd worden doorgevoerd, maar goed!). 

Wat het in de eerste 3 jaar betekent kan ik alleen maar gokken. Wellicht heeft het iets te maken met de reactietermijn. Netzoals bij die terinlageleggingen dat de documenten maar een maand of 2 beschikbaar zijn en wellicht is dat deze termijn. Maar goed vrij vaag dus!

In [25]:
result_end_date = df[pd.notnull(df['foi_endDate'])]

# Calculate the time delta in days
result_end_date['time_delta'] = (pd.to_datetime(result_end_date['foi_endDate']) - pd.to_datetime(result_end_date['foi_startDate'])).dt.days

result_end_date.groupby('dc_date_year')['time_delta'].mean()

C:\Users\ramon\AppData\Local\Temp\ipykernel_7124\3340103691.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_end_date['time_delta'] = (pd.to_datetime(result_end_date['foi_endDate']) - pd.to_datetime(result_end_date['foi_startDate'])).dt.days


dc_date_year
2013      42.451829
2014      42.618557
2015      40.718863
2017     574.666667
2018     736.777778
2019     278.090909
2020     278.857143
2021     428.928571
2022     400.066667
2023    1060.461538
Name: time_delta, dtype: float64

En inderdaad precies wat ik al dacht. Het gemiddeld aantal dagen tussen de start en einddatum ligt vele malen lager in de eerste drie jaar. Waar het dus vermoedelijk de reactietermijn is. Terwijl het hierna voornamelijk te maken heeft met de regelingen. Ik had zelf verwacht dat de time deltas een stuk hoger zouden ligge bij de eerste jaren vanaf 2017 en dan later steeds lager zouden worden. Maar bij 2023 is dit wel heel erg hoog. Dus wellicht staat deze informatie niet alleen bij de regelingen zelf maar ook bij andere documenten die beinvloed worden door een regeling, waarbij een regeling uit 2017 wellicht geldde die dan in 2023 geeindigd is ofzoiets.